In [ ]:
import os
import gzip
import shutil
from pathlib import Path

# Global variable - directory where cfg.json and .dot files generated by our code will be stored 
part1_output_directory = "output/part_1/"
part2_output_directory = "output/part_2/"

# Utility functions taken from TP1
def get_json_files(extension, directory):
   directory = Path(directory)
   return [str(file) for file in directory.rglob(extension)]

def create_output_file(filename, directory):
    # Check if output directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Check if output file already exists, if so, delete and create new file
    file_path = os.path.join(directory, filename)
    if os.path.exists(file_path):
        os.remove(file_path)

    # Open in "append" mode to avoid overwriting the whole file after each modification
    return open(directory + filename, "a")

def create_fragment_output_file(file_path: str, content: str):
    # Check if output directory exists, if not, create it
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Delete the file if it exists
    # create the file using append

    # Create the file
    with open(file_path, "a") as file:
        # You can optionally write some content to the file here
        file.write(content)

def close_output_file(file):
   file.close()

# The following function has been adapted from the following sources:
## https://www.tutorialspoint.com/How-to-scan-through-a-directory-recursively-in-Python
## https://bito.ai/resources/unzip-gz-file-python-python-explained/
## https://stackoverflow.com/questions/42445831/python-3-creating-files-in-relative-directories
def extract_zipped_files(source_directory, target_directory):
    # Check if target directory exists, if not, create it
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    # Use os.walk to recursively visit the source folder
    for current_dir, dir_names, file_names in os.walk(source_directory):
        for filename in file_names:
            # Find all zipped files
            if filename.endswith('.gz'):
                # Construct source path from source directory and the filename
                source_path = os.path.join(current_dir, filename)

                # Construct target path by replacing the source directory with target directory
                target_rel_path = os.path.relpath(current_dir, source_directory)
                target_dir = os.path.join(target_directory, target_rel_path)
                target_path = os.path.join(target_dir, filename[:-3])
                
                # Check if target directory exists, if not, create it (this is for the target directories within the main directory)
                if not os.path.exists(target_dir):
                    os.makedirs(target_dir)
                
                # Open and extract zipped files
                with gzip.open(source_path, 'rb') as f_in:
                    with open(target_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)


In [ ]:
import numpy

def __compute_manhatan_distance(vector_a, vector_b):
    # Assuming we are receiving a numpy.array type datastructure (better than iterating over each element, substracting then doing the sum)
    return numpy.abs(vector_b - vector_a).sum()

def is_similar_vector(vector_a, vector_b, similarity_treshold):
    return __compute_manhatan_distance(vector_a, vector_b) <= similarity_treshold * numpy.sum(vector_a)

In [ ]:
# Prepare output file and reader
from code_analysis import ASTReader
reader = ASTReader ()

# Prepare output arrays (we decided agains using a dictionnary for efficiency, a dict with 900 elements seemed to cause the program to be slower)
## We assumed that since we are populating these arrays at the same time and are not modifying them after, the same index in both arrays should
## point to the same file.
filename_array = []
vector_array = []

def is_identical_kit(first_path: str, second_path: str) -> bool:
    first_dir = filename_array[first_path].split('/')[:3]
    second_dir = filename_array[second_path].split('/')[:3]
    return first_dir == second_dir

def find_similar_files(directory, min_nodes, similarity_treshold, enable_identical_skip=False):
    # Prepare output file
    part1_output_directory = "output/part_1/"
    # part_1_output_file = create_output_file("part_1_output_file.txt", part1_output_directory)

    # Retrieve filenames of all ast in the specified directory
    astFilenames = get_json_files('*.json', directory)

    # Iterate over the filenames array once to visit all ast and retrieve all those with nodes > 100
    for filename in astFilenames:
        # Load ast in memory
        ast = reader.read_ast(filename)

        # Only take ASTs with more than 100 nodes into consideration
        if len(ast.get_node_ids()) > min_nodes:
            vector = ast.vectorize()
            filename_array.append(filename)
            vector_array.append(vector)  


    # Compare all vectors between themselves with using manhattan distance to find the similar ones (where MD < 0.3 in part 1)
    for v_a in range(len(vector_array)):
        for v_b in range(v_a + 1, len(vector_array)):

            # enable_identical_skip = True: compare all files
            # enable_identical_skip = False: if identical vectors are found within the same kit, skip the similarity check
            if enable_identical_skip and is_identical_kit(v_a, v_b):
                continue

            if is_similar_vector(vector_array[v_a], vector_array[v_b], similarity_treshold):
                print(f"File {filename_array[v_a]} is similar to {filename_array[v_b]}")

            # Compare all + compute manhattan distance with threshhold
            # Ignore duplicated inside the same kit
            # 
    


    # close_output_file(part_1_output_file)
            
# source_dir_part_1 = './ast'
# target_dir_part_1 = './output/extracted_ast'
# extract_zipped_files(source_dir_part_1, target_dir_part_1)
# find_similar_files(target_dir_part_1, 100, 0.3)
# find_similar_files(target_dir, 100, 0.3, enable_identical_skip=True)

In [ ]:
import tempfile
from collections import defaultdict
from collections.abc import Iterable
from copy import deepcopy
from pathlib import Path
import copy
import os

from code_analysis import CFG, ASTReader, CFGReader, Graph, AST_fragmentation
from graphviz import Source
from IPython.display import Image, display
from pydantic import BaseModel

part2_output_directory = "./output/extracted_fragments/"

def output_fragments(source_directory, target_directory):
    # Check if target directory exists, if not, create it
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    # Use os.walk to recursively visit the source folder
    for current_dir, dir_names, file_names in os.walk(source_directory):
        for filename in file_names:
            # Find all zipped files
            if filename.endswith('.ast.json'):
                # Construct source path from source directory and the filename
                source_path = os.path.join(current_dir, filename)

                # Construct target path by replacing the source directory with target directory
                target_rel_path = os.path.relpath(current_dir, source_directory)
                target_dir = os.path.join(target_directory, target_rel_path)
                target_path = os.path.join(target_dir, filename[:-3])
                
                # Check if target directory exists, if not, create it (this is for the target directories within the main directory)
                if not os.path.exists(target_dir):
                    os.makedirs(target_dir)
                
                # Write the extracted fragments in their own ast.json file
                ast = ASTReader().read_ast(source_path)
                fragments = AST_fragmentation(ast)

                for idx, fragment in enumerate(fragments):
                    output_filename = f"{filename}_{idx}.fragment.json"
                    output_filepath = os.path.join(target_dir, output_filename)
                    ast_copy = copy.deepcopy(ast)
                    ast_copy.set_root(fragment)
                    create_fragment_output_file(output_filepath, ast_copy.to_json())

#output_fragments("./output/extracted_ast/", target_directory=part2_output_directory)
source_dir_part_2 = './output/extracted_fragments'
find_similar_files(source_dir_part_2, 10, 0.1, enable_identical_skip=True)

In [ ]:
# Calculer pour chaque kit un vecteur le representant, en sommant (par colonne) tout les fragments qui composent ce kit.
def get_parametric_kits(dir_path: str):
    # Get all files and directories in the specified directory
    all_entries = os.listdir(dir_path)
    ast_reader = ASTReader()
    # Filter out only directories
    directories = [entry for entry in all_entries if os.path.isdir(os.path.join(dir_path, entry))]

    for directory in directories:
        print(directory)
        # List to store the files
        

        # Walk through the directory
        for root, dirs, files in os.walk(f"{dir_path}/{directory}"):
            for file in files:
                if file.endswith('.fragment.json'):
                    fragment_file_path = os.path.join(root, file)

                    ast = ast_reader.read_ast(fragment_file_path)
                    vector = ast.vectorize() # Represents the vector of a fragment



        # Print the list of .fragment.json files
        for file in fragment_files:
            print(file)


get_parametric_kits(source_dir_part_2)